### Query MySQL for data

In [ ]:
import pymysql

# Connect to the database
connection = pymysql.connect(host='127.0.0.1',
                             user='root',
                             port=3306,
                             db='db',
                             password='password')

cursor = connection.cursor()

print('user table size: ' + str(cursor.execute("SELECT * FROM db.users")))
print('statuses table size: ' + str(cursor.execute("SELECT * FROM db.statuses")))
print('edgelist table size: ' + str(cursor.execute("SELECT * FROM db.edgelist")))

user table size: 1090
statuses table size: 208585


## Find the Most Liked Tweets

In [ ]:
import pandas as pd
import network_vis_helper as visnet
import time
from datetime import timedelta
pd.set_option('display.max_colwidth', None)

start_time = time.time()
statuses = pd.read_sql("SELECT * FROM statuses", connection)
most_liked_tweets = (statuses[['author_screen_name', 'text', 'like_count']].
                     sort_values('like_count', ascending = False).
                     head(10))
print("--- %s time elapsed ---" % str(timedelta(seconds=time.time() - start_time)))
most_liked_tweets

Now compare to the speed of asking the same question using SQL.

In [ ]:
start_time = time.time()
most_liked_tweets = pd.read_sql("SELECT author_screen_name, text, like_count FROM statuses ORDER BY like_count DESC LIMIT 10", connection) 
print("--- %s time elapsed ---" % str(timedelta(seconds=time.time() - start_time)))
most_liked_tweets

### Visualize networks

In [ ]:
el = pd.read_sql("SELECT * FROM edgelist WHERE edge_type = 'hashtag'", connection)
weighted_el = visnet.get_weighted_el(el)
visnet.render_graph(weighted_el, edge_type = 'hashtag', edge_weight_threshold = 20)

In [ ]:
top_hashtags = (weighted_el[weighted_el['edge_type'] == 'hashtag']
                .groupby('target').sum().reset_index()
                .sort_values('weight', ascending = False))
top_hashtags.head(20)